In [ ]:
from exm.stitching.tileset import Tileset
from exm.utils import display_img as di 

In [ ]:
example_nd2 = '/mp/nas2/ruihan/20221017_alignment/20221017_alignment_slice1_final.nd2'
stitching_result = '/mp/nas3/fixstars/yves/zebrafish_data/20221017_alignment/stitching/slice1.xml'

### init from ND2 files

See the notebook Tilesets-01 for more details on ND2

### `Tileset`'s structure

Tilesets are composed of tiles accessible through the list `ts.tiles`. Each tile contains a `.img` which is a numpy array containing the image of the tile in the ZYX ordering and a `.offset` which contains a XYZ vector indicating the displacement of this tile from an origin places on the top left corner of the reconstructed volume.

It is possible to access these fields directly to write and read them.

### Init from TIFF files

TIFF files are uncompressed raw images. They do not contain any offset data so we need to initialize offsets from a different source: a ND2 file or an XML file

In [ ]:
import os

tiffdir = "/mp/nas3/fixstars/yves/zebrafish_data/20221128/segmentation"
allfiles = os.listdir(tiffdir)
fl = [f"{tiffdir}/{f}" for f in allfiles if f.startswith("seg_fov") and f.endswith(".tif")]
fl = sorted(fl)
# for fn in fl:
#     print(fn)

In [ ]:
ts=Tileset([0.1625*2, 0.1625*2, 0.4])
ts.init_from_tiff_files(fl, downscale=[1,1,1], progress=True)

In [ ]:
ts.update_offsets("/mp/nas3/fixstars/yves/zebrafish_data/20221128/stitching/example_output.xml")

In [ ]:
di(ts.show_slice(80))

### Init from JPEG 2000 files

Much lighter than TIFF files and potentially lossless, are JPEG 2000 files. We access them through the `glymur` library. Here again, they do not contain offset information (though it should be possible to hide some in the files metadata) so we need to provide offsets through a different mean. Here we use a `.npy` file that contains the meta data needed: original image sizes and offsets.

In [ ]:
jp2dir = "/mp/nas3/fixstars/yves/zebrafish_data/20221017_alignment/compression/2023-01"
allfiles = os.listdir(jp2dir)
fl = [f"{jp2dir}/{f}" for f in allfiles if f.startswith("20221017_alignment_slice1_final.nd2_fov") and f.endswith(".jp2")]
fl = sorted(fl)
# for fn in fl:
#     print(fn)

In [ ]:
ts=Tileset([0.1625*2, 0.1625*2, 0.4])
ts.init_from_jp2(fl, f"{jp2dir}/../20221017_alignment_slice1_final.nd2_offsets.npy", downscale=[4,4,2])

In [ ]:
di(ts.show_slice(50).T)

### Init from H5/BFV files

It is also possible to initialize directly from the H5/XML files produced for and by Fiji. You can pass either the `.h5` or the `.xml` file as argument but it will expect them to be in the same directory.

In [ ]:
ts=Tileset([0.1625*2, 0.1625*2, 0.4])
ts.init_from_bdv("/mp/nas3/fixstars/yves/zebrafish_data/20221128/segmentation/segmentation.h5")

In [ ]:
di(ts.show_slice(80))

### Init from H5 files

Finally there is a possibility to load tiles directly from non-BDV H5 files. These are raw tiles with no offset information and require an external source for offsets. This was only used to read segmentation results from CellPose.

In [ ]:
ts=Tileset([0.1625*2, 0.1625*2, 0.4])
ts.init_from_h5("/mp/nas2/mansour/20221017_alignment_masks/20221017_alignment_slice1_masks.h5", 
                downscale=[5,5,2], progress=True)

In [ ]:
ts.update_offsets("/mp/nas3/fixstars/yves/zebrafish_data/20221017_alignment/stitching/slice1.xml")

In [ ]:
di(ts.show_slice(50).T)